# Buiding Fast Queries on a CSV about Laptops

The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also process that data to make those queries run faster.

Here are some queries that we will want to answer:

- Given a laptop id, find the corresponding data.
- Given an amount of money, find whether there are two laptops whose total price is that given amount.
- Identify all laptops whose price falls within a given budget.

In [22]:
from csv import reader
import time
import random

In [2]:
opened_file = open('laptops.csv')
reader_file = reader(opened_file)
laptops = list(reader_file)
header = laptops[0]
laptops = laptops[1:]
opened_file.close()

In [5]:
print(header)
laptops[:2]

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


[['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898']]

# 1. Creating the class

In [6]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])

In [8]:
filename = 'laptops.csv'
laps = Inventory(filename)
print(laps.header)
print(len(laps.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## 1.1 Method 1: Looking up a laptop from a given ID

### 1.1.1 Algorithm a: Looping through the dataset

In [9]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None

In [14]:
laps = Inventory('laptops.csv')
laps.get_laptop_from_id('3362737')

['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [15]:
print(laps.get_laptop_from_id('3362736'))

None


### 1.1.2 Algorithm b: Implement a dictionary instead

In [16]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

In [18]:
laps = Inventory('laptops.csv')
laps.get_laptop_from_id_fast('3362737')

['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [20]:
print(laps.get_laptop_from_id_fast('3362736'))

None


### 1.1.3 Comparing 2 implementations

In [32]:
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

In [35]:
print(type(ids[0]))

<class 'str'>


In [38]:
laps = Inventory('laptops.csv')
total_time_no_dict = 0
total_time_dict = 0

for idx in ids:
    start = time.time()
    laps.get_laptop_from_id(idx)
    end = time.time()
    elapsed_time = end - start
    total_time_no_dict += elapsed_time

for idx in ids:
    start = time.time()
    laps.get_laptop_from_id_fast(idx)
    end = time.time()
    elapsed_time = end - start
    total_time_dict += elapsed_time

print('Total time using loops:', total_time_no_dict)
print('Total time using dictionary:', total_time_dict)

Total time using loops: 0.7306997776031494
Total time using dictionary: 0.0040378570556640625


The new way is about 183 times faster than the original implementation for this input size (10000)

## 2. Writing a function that helps determine whether a gift card is valid (Valid gift card means the customers can spend it in full)
### Either there is one laptop that costs exactly as the gift card
### Or there are two laptops whose costs sum up to the gift card's value

In [39]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for rowa in self.rows:
            for rowb in self.rows:
                if rowa[-1] + rowb[-1] == dollars:
                    return True
        return False

In [40]:
laps = Inventory('laptops.csv')
print(laps.check_promotion_dollars(1000))
print(laps.check_promotion_dollars(442))

True
False


## 2.1 Improving the algorithm

In [41]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for rowa in self.rows:
            for rowb in self.rows:
                if rowa[-1] + rowb[-1] == dollars:
                    return True
        return False
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        return False

In [42]:
laps = Inventory('laptops.csv')
print(laps.check_promotion_dollars_fast(1000))
print(laps.check_promotion_dollars_fast(442))

True
False


### 2.2 Comparing two implementations

In [43]:
prices = [(random.randint(100, 5000)) for _ in range(100)]

In [44]:
laps = Inventory('laptops.csv')
total_time_no_set = 0
total_time_set = 0

for price in prices:
    start = time.time()
    laps.check_promotion_dollars(price)
    end = time.time()
    elapsed_time = end - start
    total_time_no_set += elapsed_time

for price in prices:
    start = time.time()
    laps.check_promotion_dollars_fast(price)
    end = time.time()
    elapsed_time = end - start
    total_time_set += elapsed_time

print('Total time using loops:', total_time_no_set)
print('Total time using set:', total_time_set)

Total time using loops: 0.8062057495117188
Total time using set: 0.0003662109375


Using set is significantly faster that using loops by roughly 2687 times for this input size

# 3. Finding the first laptop that is more expensive than a given price

We will implement a Binary search to find the first laptop that costs more than a specific price amount

In [45]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            read_file = reader(f)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        self.id_to_row = {}
        self.prices = set()
        #Converting price of each row to an integer (last column)
        for row in self.rows:
            row[-1] = int(row[-1])
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=lambda row:row[-1])
        
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for rowa in self.rows:
            for rowb in self.rows:
                if rowa[-1] + rowb[-1] == dollars:
                    return True
        return False
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if (dollars - price) in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        start = 0
        end = len(self.rows_by_price) - 1
        while start < end:
            mid = (start + end) // 2
            price = self.rows_by_price[mid][-1]
            if price <= target_price:
                start = mid + 1
            else:
                end = mid
        if self.rows_by_price[start][-1] <= target_price:
            return -1
        return self.rows_by_price[start]

In [47]:
laps = Inventory('laptops.csv')                              
print(laps.find_first_laptop_more_expensive(1000), '\n') 
print(laps.find_first_laptop_more_expensive(10000), '\n')

['8747948', 'Lenovo', 'ThinkPad T460', 'Notebook', '14', '1366x768', 'Intel Core i5 6200U 2.3GHz', '4GB', '508GB Hybrid', 'Intel HD Graphics 520', 'Windows 7', '1.70kg', 1002] 

-1 



We found a laptop that costs more than 1000 dollars which is the Lenovo ThinkPad T460. However, no laptop matches the price \$10000 